In [ ]:
%matplotlib inline

# Scalable problem.

We want to solve the Aerostructure MDO problem
by means of the [MDF][gemseo.formulations.mdf.MDF] formulation
with a higher dimension for the sweep parameter.
For that, we use the [ScalableProblem][gemseo.problems.mdo.scalable.data_driven.problem.ScalableProblem] class.


In [ ]:
from __future__ import annotations

from gemseo import create_discipline
from gemseo import create_scenario
from gemseo.problems.mdo.aerostructure.aerostructure_design_space import (
    AerostructureDesignSpace,
)
from gemseo.problems.mdo.scalable.data_driven.problem import ScalableProblem

## Define the design problem

In a first step, we define the design problem in terms of
objective function (to maximize or minimize),
design variables (local and global)
and constraints (equality and inequality).



In [ ]:
design_variables = ["thick_airfoils", "thick_panels", "sweep"]
objective_function = "range"
eq_constraints = ["c_rf"]
ineq_constraints = ["c_lift"]
maximize_objective = True

## Create the disciplinary datasets

Then, we create the disciplinary [BaseFullCache][gemseo.caches.base_full_cache.BaseFullCache] datasets
based on a [DiagonalDOE][gemseo.algos.doe.diagonal_doe.diagonal_doe.DiagonalDOE].



In [ ]:
disciplines = create_discipline(["Aerodynamics", "Structure", "Mission"])
datasets = []
for discipline in disciplines:
    design_space = AerostructureDesignSpace()
    design_space.filter(discipline.io.input_grammar.names)
    output_names = iter(discipline.io.output_grammar.names)
    scenario = create_scenario(
        discipline,
        next(output_names),
        design_space,
        formulation_name="DisciplinaryOpt",
        scenario_type="DOE",
    )
    for output_name in output_names:
        scenario.add_observable(output_name)
    scenario.execute(algo_name="DiagonalDOE", n_samples=10)
    datasets.append(scenario.to_dataset(name=discipline.name, opt_naming=False))

## Instantiate a scalable problem

In a third stage, we instantiate a [ScalableProblem][gemseo.problems.mdo.scalable.data_driven.problem.ScalableProblem]
from these disciplinary datasets and from the definition of the MDO problem.
We also increase the dimension of the sweep parameter.



In [ ]:
problem = ScalableProblem(
    datasets,
    design_variables,
    objective_function,
    eq_constraints,
    ineq_constraints,
    maximize_objective,
    sizes={"sweep": 2},
)

!!! note

    We could also provide options to the [ScalableModel][gemseo.problems.mdo.scalable.data_driven.model.ScalableModel] objects
    by means of the constructor of [ScalableProblem][gemseo.problems.mdo.scalable.data_driven.problem.ScalableProblem],
    e.g. `fill_factor` in the frame of the [ScalableDiagonalModel][gemseo.problems.mdo.scalable.data_driven.diagonal.ScalableDiagonalModel].
    In this example, we use the standard ones.



## Visualize the N2 chart

We can see the coupling between disciplines through this N2 chart:



In [ ]:
problem.plot_n2_chart(save=False, show=True)

## Create an MDO scenario

Lastly, we create an [MDOScenario][gemseo.scenarios.mdo_scenario.MDOScenario] with the [MDF][gemseo.formulations.mdf.MDF] formulation
and start the optimization at equilibrium,
thus ensuring the feasibility of the first iterate.



In [ ]:
scenario = problem.create_scenario("MDF", start_at_equilibrium=True)

!!! note

    We could also provide options for the scalable models to the constructor
    of [ScalableProblem][gemseo.problems.mdo.scalable.data_driven.problem.ScalableProblem],
    e.g. `fill_factor` in the frame of
    the [ScalableDiagonalModel][gemseo.problems.mdo.scalable.data_driven.diagonal.ScalableDiagonalModel].
    In this example, we use the standard ones.



Once the scenario is created, we can execute it as any scenario.
Here, we use the `NLOPT_SLSQP` optimization algorithm
with no more than 100 iterations.



In [ ]:
scenario.execute(algo_name="NLOPT_SLSQP", max_iter=100)

We can post-process the results.
Here, we use the standard [OptHistoryView][gemseo.post.opt_history_view.OptHistoryView].



In [ ]:
scenario.post_process(post_name="OptHistoryView", save=False, show=True)